In [2]:
import pandas as pd
import numpy as np

# Strip Home Office Data (keep sheets separated)

In [ ]:
# Excel-Datei laden
datei = "./Data/Home_Office_Numbers.xlsx"
xls = pd.read_excel(datei, sheet_name=None, header=None)  # Keine Header einlesen

neue_sheets = {}
neuer_name = "Region"
exclude = {"zusammenfassung", "struktur"}

for name, df in xls.items():
    # Sheets mit "Kennzeichen" im Namen überspringen
    if name.lower() in exclude:
        continue

    if "kennzeichen" in name.lower():
        continue

    # Kopf-Infos aus Zeile 6–8 (Index 5–7), Spalte 3 (Index 2)
    if df.shape[0] >= 8 and df.shape[1] >= 3:
        geschlecht = str(df.iloc[5, 2]).strip()
        altersklasse = str(df.iloc[6, 2]).strip()
        kinder = str(df.iloc[7, 2]).strip()
    else:
        geschlecht = altersklasse = kinder = None

    # Datenbereich ab Zeile 9 (Index 8)
    df = df.iloc[10:].reset_index(drop=True)

    df = df.drop(index=0)  # Zeilen-Index beginnt bei 0 -> index=1 ist die zweite Zeile

    # die letzten 3 Zeilen löschen
    df = df[:-3]

    df.columns = df.iloc[0] 
    df = df[1:].reset_index(drop=True) 

    # Neue Spalten einfügen
    df["Geschlecht"] = geschlecht
    df["Altersklasse"] = altersklasse
    df["Kinder"] = kinder
    
    # Erste Spalte umbenennen
    alte_spalten = df.columns.tolist()
    if alte_spalten:
        alte_spalten[0] = "Region"
        df.columns = alte_spalten

    df = df.drop(index=0)

    df = df.iloc[:37].reset_index(drop=True)

    neue_sheets[name] = df.reset_index(drop=True)  # Optional: Index zurücksetzen

# Ergebnis speichern (optional)
with pd.ExcelWriter("Home_Office_stripped.xlsx", engine="openpyxl") as writer:
    for name, df in neue_sheets.items():
        df.to_excel(writer, sheet_name=name, index=False)

# Strip Home Office Data (CONCAT)

In [ ]:
# Excel-Datei laden
datei = "./Data/Home_Office_Numbers.xlsx"
xls = pd.read_excel(datei, sheet_name=None, header=None)  # Keine Header einlesen

neue_sheets = {}
neuer_name = "Region"
exclude = {"zusammenfassung", "struktur"}
all_sheets = []  # Liste zum Sammeln aller DataFrames

for name, df in xls.items():
    # strip off every sheet which includes "Kennzeichen", "zusammenfassung", "struktur"
    if name.lower() in exclude:
        continue
    if "kennzeichen" in name.lower():
        continue

    # head-info from line 6–8 (Index 5–7), column 3 (Index 2)
    if df.shape[0] >= 8 and df.shape[1] >= 3:
        geschlecht = str(df.iloc[5, 2]).strip()
        altersklasse = str(df.iloc[6, 2]).strip()
        kinder = str(df.iloc[7, 2]).strip()
    else:
        geschlecht = altersklasse = kinder = None

    # data beginning from line no. 9 (Index 8)
    df = df.iloc[10:].reset_index(drop=True)

    df = df.drop(index=0)  # Zeilen-Index beginnt bei 0 -> index=1 ist die zweite Zeile

    # delete last 3 rows (includes only footnotes)
    df = df[:-3]

    df.columns = df.iloc[0] 
    df = df[1:].reset_index(drop=True) 

    # Insert new columns
    df["Geschlecht"] = geschlecht
    df["Altersklasse"] = altersklasse
    df["Kinder"] = kinder
    
    # Rename first column
    alte_spalten = df.columns.tolist()
    if alte_spalten:
        alte_spalten[0] = "Region"
        df.columns = alte_spalten

    # erste Zeile löschen
    df = df.drop(index=0)

    # clear all footnotes –> keep only lines 1 to 37 
    df = df.iloc[:37].reset_index(drop=True)

    neue_sheets[name] = df.reset_index(drop=True) 

    all_sheets.append(df)

# concat all data into one sheet
gesamt_df = pd.concat(all_sheets, ignore_index=True)

# Safe new data as excel-file (.xlsx)
with pd.ExcelWriter("Home_Office_all_sheets.xlsx", engine="openpyxl") as writer:
    gesamt_df.to_excel(writer, sheet_name="all_sheets", index=False)

# Simplify and translate data into english

In [7]:
import pandas as pd

In [5]:
# Load Excel-File
df = pd.read_excel('./Home_Office_all_sheets.xlsx')

In [ ]:
# Change column names
df.rename(columns={
    'Region': 'region',
    'Geschlecht': 'gender',
    'Altersklasse': 'age_group',
    'Kinder' : 'children'
}, inplace=True)

In [ ]:
# Change values in columns 'region'
df['region'] = df['region'].replace({
    'Europäische Union - 27 Länder (ab 2020)': 'European_Union_27_countries_from_2020',
    'Euroraum - 20 Länder (ab 2023)': 'Euro_area_20_countries_2015-2022',
    'Belgien': 'Belgium',
    'Bulgarien': 'Bulgaria',
    'Tschechien': 'Czechia',
    'Dänemark': 'Denmark',
    'Deutschland': 'Germany',
    'Estland': 'Estonia',
    'Irland': 'Ireland',
    'Griechenland': 'Greece',
    'Spanien': 'Spain',
    'Frankreich': 'France',
    'Kroatien': 'Croatia',
    'Italien': 'Italy',
    'Zypern': 'Cyprus',
    'Lettland': 'Latvia',
    'Litauen': 'Lithuania',
    'Luxemburg': 'Luxembourg',
    'Ungarn': 'Hungary',
    'Malta': 'Malta',
    'Niederlande': 'Netherlands',
    'Österreich': 'Austria',
    'Polen': 'Poland',
    'Portugal': 'Portugal',
    'Rumänien': 'Romania',
    'Slowenien': 'Slovenia',
    'Slowakei': 'Slovakia',
    'Finnland': 'Finland',
    'Schweden': 'Sweden',
    'Norwegen': 'Norway',
    'United Kingdom': 'United Kingdom',
    'Bosnien und Herzegowina': 'Bosnia and Herzegovina',
    'Montenegro': 'Montenegro',
    'Nordmazedonien': 'North Macedonia',
    'Serbien': 'Serbia',
    'Türkei': 'Türkiye',
})

#Change values in columns 'gender'
df['gender'] = df['gender'].replace({
    'Insgesamt': 'total',
    'Männer': 'male',
    'Frauen': 'female'
})

# Change values in columns 'age_group'
df['age_group'] = df['age_group'].replace({
    '18 bis 24 Jahre': '18-24',
    '18 bis 64 Jahre': '18-64',
    '18 Jahre und mehr': '18+',
    '20 bis 49 Jahre': '20-49',
    '25 bis 49 Jahre': '25-49',
    '25 bis 54 Jahre': '25-54',
    '55 Jahre und mehr': '55+',
    '18 bis 24 Jahre': '18-24',
    '18 bis 64 Jahre': '18-64',
    '18 Jahre und mehr': '18+'
})

# Change values in columns 'children'
df['children'] = df['children'].replace({
    'Insgesamt': 'total',
    'Keine Kinder': '0',
    '1 Kind': '1',
    '2 Kinder': '2',
    '3 Kinder und mehr': '3+'
})

# Safe new data as excel-file (.xlsx)
df.to_excel('Home_Office_all_sheets_english.xlsx', index=False)